- connect campaign
- change model parameters

In [1]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import preprocessing
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.metrics import average_precision_score
tqdm_notebook.pandas(desc="progress: ")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  
/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
DIR = "../input/20201115at/"
train = pd.read_csv(DIR+"train.csv")
test = pd.read_csv(DIR+"test.csv")
campaign = pd.read_csv(DIR+"campaign.csv")

#advertiser_video = pd.read_csv(DIR+"advertiser_video.csv")
#map_game_feed_native_video_assets = pd.read_csv(DIR+'map_game_feed_native_video_assets.csv')
#advertiser_converted_video = pd.read_csv(DIR+"advertiser_converted_video.csv")

# preprocess

In [3]:
train = train.merge(campaign, left_on="campaign_id", right_on = "id", how="left").drop(["mst_advertiser_id", "id"], axis=1)
test = test.merge(campaign, left_on="campaign_id", right_on = "id", how="left").drop(["mst_advertiser_id", "id"], axis=1)


#train = merge_all(train, campaign, map_game_feed_native_video_assets, advertiser_video, advertiser_converted_video)
#test = merge_all(test, campaign, map_game_feed_native_video_assets, advertiser_video, advertiser_converted_video)

In [4]:
train["imp_at"] = pd.to_datetime(train["imp_at"])

In [5]:
train.drop(['click_or_view_at', 'cv_at','last_cv_interval', 'last_paid_interval'], axis=1, inplace=True)
train.drop(['request_id'], axis=1, inplace=True)
test.drop(['request_id'], axis=1, inplace=True)

In [6]:
object_feats = [i for i in train.columns if "object" == str(train[i].dtype)]
train[object_feats] = train[object_feats].fillna("-999")
test[object_feats] = test[object_feats].fillna("-999")

for i in object_feats:
    print(i)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(pd.concat([train[i], test[i]])))
    train[i] = lbl.transform(list(train[i]))
    test[i] = lbl.transform(list(test[i]))

advertiser_id
app_id
campaign_id
country_code
game_feed_id
game_template_id
media_app_id
os_version
uid
video_template_id
mst_advertiser_order_id


In [7]:
train.head()

,imp_at,target,adnw_id,adspot_id,adspot_video_format_id,advertiser_id,app_id,auction_type_id,campaign_id,category_id,...,max_login_interval,media_app_id,os,os_version,pos,uid,user_type_id,video_template_id,mst_advertiser_order_id,mst_user_type_id
0,2020-04-28 00:04:14.639,0,4,1111,2.0,20,14872,1.0,403,8497,...,2607.0,0,1,87,0,273693,1,6,37,35
1,2020-04-28 00:09:13.759,0,28,1081,2.0,121,8301,2.0,91,0,...,13540.0,8995,2,152,1,427139,2,4,166,37
2,2020-04-28 00:09:14.708,0,42,1123,3.0,5,12948,1.0,114,0,...,33725.0,347,1,55,0,1005266,1,0,27,35
3,2020-04-28 00:10:40.291,0,8,1113,2.0,22,12379,1.0,75,0,...,7377.0,6145,1,91,0,492541,1,7,159,38
4,2020-04-28 00:11:45.895,0,22,1061,NaN,135,196,4.0,331,0,...,164759.0,2190,1,71,0,698493,1,0,56,35


# fe

In [8]:
#train["type"] = "train"
#test["type"] = "test"
#all_df = pd.concat([train, test])

In [9]:
# cumsum from click by uid
#tmp1 = all_df.groupby(['uid'])['from_click'].progress_apply(lambda x: pd.DataFrame([0]+list(x.cumsum()[:-1]))).reset_index(drop=True).values
#tmp1 = np.array(tmp1, dtype=float)
# cumsum total num by uid
#tmp2 = all_df.groupby(['uid']).cumcount().values
#tmp2 = np.array(tmp2, dtype=float).reshape(-1,1)
# https://stackoverflow.com/questions/26248654/how-to-return-0-with-divide-by-zero
#from_click_ratio = np.divide(tmp1, tmp2, out=np.ones_like(tmp1)*-1, where=tmp2!=0)
#all_df["click_ratio"] = from_click_ratio

In [10]:
#train = all_df[all_df.type="train"].reset_index(drop=True)
#test = all_df[all_df.type="test"].reset_index(drop=True)

# separation for validation

In [11]:
mask = train.imp_at > "2020-05-21"
valid = train.loc[mask].copy()
train = train.loc[~mask]

In [12]:
remove_features = ["imp_at"]
    
#for i in X_train.columns:
#    if (X_train[i].std() == 0) and i not in remove_features:
#        remove_features.append(i)
    
train = train.drop(remove_features, axis=1)
valid = valid.drop(remove_features, axis=1)
test = test.drop(remove_features, axis=1)

In [13]:
train.shape, valid.shape

((1568715, 31), (428880, 31))

In [14]:
train.head()

,target,adnw_id,adspot_id,adspot_video_format_id,advertiser_id,app_id,auction_type_id,campaign_id,category_id,country_code,...,max_login_interval,media_app_id,os,os_version,pos,uid,user_type_id,video_template_id,mst_advertiser_order_id,mst_user_type_id
0,0,4,1111,2.0,20,14872,1.0,403,8497,1,...,2607.0,0,1,87,0,273693,1,6,37,35
1,0,28,1081,2.0,121,8301,2.0,91,0,1,...,13540.0,8995,2,152,1,427139,2,4,166,37
2,0,42,1123,3.0,5,12948,1.0,114,0,1,...,33725.0,347,1,55,0,1005266,1,0,27,35
3,0,8,1113,2.0,22,12379,1.0,75,0,1,...,7377.0,6145,1,91,0,492541,1,7,159,38
4,0,22,1061,NaN,135,196,4.0,331,0,1,...,164759.0,2190,1,71,0,698493,1,0,56,35


# modelling

In [15]:
lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt',
               'tree_learner': 'serial', 'learning_rate': 0.1, "num_leaves": 10, 'random_seed':44,'max_depth': 5}

def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

y_train = train.target.copy()
X_train = train.drop(['target'],axis=1).copy()
y_valid = valid.target.copy()
X_valid = valid.drop(['target'],axis=1).copy()
    
valid_lgb = pd.DataFrame(np.zeros([X_train.shape[0]]))
real = np.array([])
features_list = [i for i in X_train.columns]
feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
        
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        
model = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
       num_boost_round=10000,early_stopping_rounds=20,verbose_eval = 10, feval=pr_auc_metric, categorical_feature = sorted(object_feats))
        
valid_predict = model.predict(X_valid, num_iteration = model.best_iteration)
feature_importance_df["Importance"] = model.feature_importance()
        
#feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
#feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
#feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

score = average_precision_score(y_valid, valid_predict)
print("average precision score = {}".format(score))
pred_value = model.predict(test, num_iteration = model.best_iteration)

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['advertiser_id', 'app_id', 'campaign_id', 'country_code', 'game_feed_id', 'game_template_id', 'media_app_id', 'mst_advertiser_order_id', 'os_version', 'uid', 'video_template_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 20 rounds
[10]	training's pr_auc: 0.214462	valid_1's pr_auc: 0.177802
[20]	training's pr_auc: 0.221781	valid_1's pr_auc: 0.181468
[30]	training's pr_auc: 0.228937	valid_1's pr_auc: 0.188282
[40]	training's pr_auc: 0.237917	valid_1's pr_auc: 0.192951
[50]	training's pr_auc: 0.24398	valid_1's pr_auc: 0.195934
[60]	training's pr_auc: 0.249427	valid_1's pr_auc: 0.198244
[70]	training's pr_auc: 0.253702	valid_1's pr_auc: 0.199983
[80]	training's pr_auc: 0.257764	valid_1's pr_auc: 0.201727
[90]	training's pr_auc: 0.260622	valid_1's pr_auc: 0.202683
[100]	training's pr_auc: 0.26323	valid_1's pr_auc: 0.203209
[110]	training's pr_auc: 0.265844	valid_1's pr_auc: 0.203871
[120]	training's pr_auc: 0.268632	valid_1's pr_auc: 0.204406
[130]	training's pr_auc: 0.270421	valid_1's pr_auc: 0.204867
[140]	training's pr_auc: 0.272262	valid_1's pr_auc: 0.205237
[150]	training's pr_auc: 0.274049	valid_1's pr_auc: 0.205878
[160]	training's pr_auc: 0.276391	v

In [16]:
feature_importance_df.sort_values("Importance", ascending=False)

,Feature,Importance
13,game_feed_id,442
4,app_id,437
25,uid,398
6,campaign_id,277
21,media_app_id,181
9,first_login_interval,117
18,last_login_interval,90
20,max_login_interval,81
3,advertiser_id,68
19,login_frequency,59


# submission

In [17]:
lgb_sub = pd.read_csv(DIR + "atmaCup7__sample_submission.csv")
lgb_sub["target"] = pred_value
lgb_sub.to_csv("atmacup7_lgb"+str(score)[:-10]+".csv", index = False)